In [2]:
import pandas as pd
import numpy as np
oppertunity = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/case_oppertunities.csv')
case_text = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_text_fact.csv')

In [3]:
print(oppertunity.columns)


Index(['Unnamed: 0', 'STATUS', 'PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT',
       'CATEGORY', 'SUBCATEGORY', 'CREATED_DATE', 'CLOSED_DATE',
       'CASE_CLOSED_REASON', 'RECORDTYPE', 'CASE_NUMBER', 'ACCOUNTKEY',
       'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL', 'EMPLOYEECOUNT', 'THEATER',
       'SUBREGION', 'SEGMENT', 'SUBSEGMENT', 'Tenure', 'Zero.Revenue.Flag',
       'Einstein Analytics', 'Role Based - Online', 'Server', 'Saas',
       'Role Based - On Prem', 'Services', 'Desktop', 'Role Based - NonProd',
       'Other', 'CLOSEDATE', 'ISWON', 'ISPARTNERRELATED'],
      dtype='object')


In [4]:
sales = oppertunity[oppertunity['ISWON'] == 1]
sales = sales.merge(case_text,
                          how='left',
                          on='CASE_NUMBER')
print(sales.shape)

(54546, 40)


In [5]:
sales['Einstein Analytics'] = np.where(sales['Einstein Analytics'] >0, 1, 0)
sales['Role Based - Online & Saas'] = np.where((sales['Role Based - Online'] >0) |
                                        (sales['Saas'] > 0), 1, 0)
sales['Server'] = np.where(sales['Server'] >0, 1, 0)
sales['Role Based - On Prem'] = np.where((sales['Role Based - On Prem'] >0) &
                                         (sales['Einstein Analytics'] == 0) &
                                         (sales['Server'] == 0) &
                                         (sales['Saas'] == 0) &
                                         (sales['Role Based - Online'] == 0) &
                                         (sales['Services'] == 0) &
                                         (sales['Desktop'] == 0) &
                                         (sales['Other'] == 0) &
                                         (sales['Role Based - NonProd'] == 0), 1, 0)
sales['Services'] = np.where(sales['Services'] >0, 1, 0)
sales['Other'] = np.where((sales['Other'] >0) |
                          (sales['Role Based - NonProd'] >0) |
                          (sales['Desktop'] >0), 1, 0)

print(sales['Role Based - On Prem'].sum() / len(sales))
print(sales['Role Based - Online & Saas'].sum() / len(sales))
print(sales['Einstein Analytics'].sum() / len(sales))
print(sales['Server'].sum() / len(sales))
print(sales['Other'].sum() / len(sales))

labels = sales[['Role Based - On Prem',
                'Einstein Analytics', 
                'Role Based - Online & Saas', 
                'Server',
                'Other']]
num_classes = len(labels.columns)
print(num_classes)

0.5862574707586258
0.1748249184174825
0.021486451802148646
0.05212114545521211
0.15462178711546218
5


In [6]:
cases = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_fact.csv')
oppertunity = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/case_oppertunities.csv')
casetext = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_text_fact.csv')
cases_by_account = cases.groupby('ACCOUNTKEY')['CASE_NUMBER'].agg(['count'])
#print(cases_by_account.head)
oppertunity = oppertunity[oppertunity['ISWON'] == 1]
oppertunity['completetion_time'] = pd.to_datetime(oppertunity['CLOSED_DATE']) - pd.to_datetime(oppertunity['CREATED_DATE'])

oppertunity = oppertunity.merge(cases_by_account,
                                on = 'ACCOUNTKEY',
                                how='left')
print(oppertunity.columns)
oppertunity['completetion_time'] = oppertunity['completetion_time'].astype('int')

Index(['Unnamed: 0', 'STATUS', 'PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT',
       'CATEGORY', 'SUBCATEGORY', 'CREATED_DATE', 'CLOSED_DATE',
       'CASE_CLOSED_REASON', 'RECORDTYPE', 'CASE_NUMBER', 'ACCOUNTKEY',
       'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL', 'EMPLOYEECOUNT', 'THEATER',
       'SUBREGION', 'SEGMENT', 'SUBSEGMENT', 'Tenure', 'Zero.Revenue.Flag',
       'Einstein Analytics', 'Role Based - Online', 'Server', 'Saas',
       'Role Based - On Prem', 'Services', 'Desktop', 'Role Based - NonProd',
       'Other', 'CLOSEDATE', 'ISWON', 'ISPARTNERRELATED', 'completetion_time',
       'count'],
      dtype='object')


/var/folders/t_/rcfcs8g56jn7trwnsvmdyh_r0000gn/T/ipykernel_53076/2069459914.py:13: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  oppertunity['completetion_time'] = oppertunity['completetion_time'].astype('int')


In [7]:
print(casetext.columns)

Index(['Unnamed: 0', 'SUBJECT', 'DESCRIPTION', 'ISSUE_SUMMARY',
       'CLOSED_CASE_NOTES', 'CASE_NUMBER'],
      dtype='object')


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
oppertunity_text = oppertunity.merge(casetext,
                                     how='left',
                                     on = 'CASE_NUMBER')
oppertunity_text['text'] = oppertunity_text['DESCRIPTION']\
    .str.replace('[^a-zA-Z]', ' ').str.lower() + oppertunity_text['ISSUE_SUMMARY']\
        .str.replace('[^a-zA-Z]', ' ').str.lower() + oppertunity_text['DESCRIPTION']\
            .str.replace('[^a-zA-Z]', ' ').str.lower()
text = oppertunity_text[['text']]
text.fillna('None', inplace=True)
tfidf = TfidfVectorizer(max_features= 1000,
                        max_df=0.5,
                        ngram_range=(1, 2),
                        stop_words='english')
text_tfidf = tfidf.fit_transform(text['text'])

/var/folders/t_/rcfcs8g56jn7trwnsvmdyh_r0000gn/T/ipykernel_53076/464594555.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  oppertunity_text['text'] = oppertunity_text['DESCRIPTION']\
/var/folders/t_/rcfcs8g56jn7trwnsvmdyh_r0000gn/T/ipykernel_53076/464594555.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[^a-zA-Z]', ' ').str.lower() + oppertunity_text['ISSUE_SUMMARY']\
/var/folders/t_/rcfcs8g56jn7trwnsvmdyh_r0000gn/T/ipykernel_53076/464594555.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[^a-zA-Z]', ' ').str.lower() + oppertunity_text['DESCRIPTION']\
/Users/michaelcolellajensen/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [62]:
text_df = pd.DataFrame(text_tfidf.toarray(), 
                       columns = tfidf.get_feature_names_out())


In [63]:
case_categories = oppertunity[['STATUS','PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT',
                                    'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL','THEATER',
                                    'SUBREGION', 'SEGMENT', 'SUBSEGMENT']]
descrete = oppertunity[['Tenure', 'completetion_time', 'count', 'EMPLOYEECOUNT']]
case_one_hot_encode = pd.get_dummies(case_categories)
case_context_features = pd.concat([case_one_hot_encode, 
                                   descrete],
                                   axis=1)

case_context_features['completetion_time'] = case_context_features['completetion_time'].astype('int')
case_features = pd.concat([case_context_features, text_df],axis=1)
print(labels.shape)
print(case_features.shape)


(54546, 5)
(54546, 1150)


In [11]:
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(case_context_features,
                                                    labels,
                                                    test_size=0.25,
                                                    stratify=labels,
                                                    random_state=123)    
xgb_model = xgb.XGBClassifier(n_estimators = 770,
                              eta = 0.105,
                              max_depth = 8,
                              max_delta_step = 4,
                              min_child_weight = 0.435,
                              min_sample_split = 0.225,
                              n_jobs = 8)
multiclass_model = OneVsRestClassifier(xgb_model)
multiclass_model.fit(x_train, y_train)
predictions = multiclass_model.predict(x_test)
print('Overall Accuracy: ' + str(multiclass_model.score(x_test, y_test)))

/Users/michaelcolellajensen/Library/Python/3.8/lib/python/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:39:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:576: 
Parameters: { "min_sample_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:39:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:40:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:576: 
Parameters: { "min_sample_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getti

In [13]:
y_class0 = y_test['Role Based - On Prem'].values
predictions0 = predictions[:,0]
class0 = np.where((y_class0 == predictions0) &
                   ((predictions0 > 0)|
                    (y_class0 > 0)), 1, 0)
class0_acc = class0.sum() / y_class0.sum()
print('Role Based - On Prem: ' + str(class0_acc))

y_class1 = y_test['Einstein Analytics'].values
predictions1 = predictions[:,1]
class1 = np.where((y_class1 == predictions1) &
                   ((predictions1 > 0)|
                    y_class1>0), 1, 0)
class1_acc = class1.sum() / y_class1.sum()
print('Einstein Analytics: ' + str(class1_acc))

y_class2 = y_test['Role Based - Online & Saas'].values
predictions2 = predictions[:,2]
class2 = np.where((y_class2 == predictions2) &
                   ((predictions2 > 0)|
                    (y_class2 > 0)), 1, 0)
class2_acc = class2.sum() / y_class2.sum()
print('Role Based - Online & Saas: ' + str(class2_acc))

y_class3 = y_test['Server'].values
predictions3 = predictions[:,3]
class3 = np.where((y_class3 == predictions3) &
                   ((predictions3 > 0)|
                    y_class3 > 0), 1, 0)
class3_acc = class3.sum() / y_class3.sum()
print('Server: ' + str(class3_acc))

y_class4 = y_test['Other'].values
predictions4 = predictions[:,4]
class4 = np.where((y_class4 == predictions2) &
                   ((predictions4 > 0)|
                    (y_class4 > 0)), 1, 0)
class4_acc = class4.sum() / y_class4.sum()
print('Other: ' + str(class4_acc))

Role Based - On Prem: 0.8537836147592245
Einstein Analytics: 0.2627986348122867
Role Based - Online & Saas: 0.6034410407049937
Server: 0.2039381153305204
Other: 0.2077798861480076


In [64]:
x_train_text, x_test_text, y_train_text, y_test_text = train_test_split(text_tfidf,
                                                                        labels,
                                                                        test_size=0.25,
                                                                        stratify=labels,
                                                                        random_state=123)    
xgb_text_model = xgb.XGBClassifier(n_estimators = 550,
                                   eta = 0.05,
                                   max_depth = 12,
                                   max_delta_step = 4,
                                   min_child_weight = 0.435,
                                   min_sample_split = 0.125,
                                   n_jobs = 8)
text_multiclass = OneVsRestClassifier(xgb_text_model)
text_multiclass.fit(x_train_text, y_train_text)
text_predictions = text_multiclass.predict(x_test_text)
print('Overall Accuracy: ' + str(text_multiclass.score(x_test_text, y_test_text)))

/Users/michaelcolellajensen/Library/Python/3.8/lib/python/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:42:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:576: 
Parameters: { "min_sample_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:42:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:44:49] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:576: 
Parameters: { "min_sample_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getti

In [101]:
dict = text_multiclass.estimators_[2].get_booster().get_score(importance_type='gain')
keys = list(dict.keys())
values = list(dict.values())
feature_df = pd.DataFrame(data=values,
                          index=keys,
                          columns= ['score'],
                          ).sort_values(by='score',
                                        ascending=False)
print(feature_df.head(20))

          score
f844  66.383354
f253  44.469250
f86   13.678738
f993  13.127463
f449  10.584758
f54    9.244251
f852   9.110429
f699   8.515123
f372   7.805783
f856   7.775749
f102   7.620206
f541   7.411538
f995   7.237294
f951   7.023158
f29    6.949738
f619   6.788813
f959   6.761335
f173   6.758111
f763   6.738438
f532   6.635305


In [105]:
feature_words = tfidf.get_feature_names()
print(feature_words[856])
print(feature_words[372])
print(feature_words[699])
print(feature_words[852])


tableausoftware tabadmin
https help
sales
tableausoftware
